In [1]:
# ============================================================================
# NOTEBOOK 1: APT41 Incident Response - Automated Remediation
# Fichier: apt41_incident_response.ipynb
# ============================================================================

"""
Ce notebook propose des solutions automatisées pour les détections APT41
Basé sur l'analyse IA et les recommandations MITRE ATT&CK
"""
# CELL 1: Configuration avec suppression des warnings
import sys
import subprocess
import warnings

# Supprimer les warnings pandas
warnings.filterwarnings('ignore', message='.*SQLAlchemy.*')

# Installer psycopg2 si nécessaire
try:
    import psycopg2
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "psycopg2-binary"])
    import psycopg2

import pandas as pd
import json
from datetime import datetime, timedelta
from IPython.display import display, Markdown, HTML

# Configuration de la base de données
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'kestrel_hunts',
    'user': 'kestrel',
    'password': 'Kestrel@2025!'
}

def connect_db():
    """Connexion à PostgreSQL"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Connected to PostgreSQL")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

conn = connect_db()
# ============================================================================
# CELL 2: Connexion à la base de données
# ============================================================================

def connect_db():
    """Connexion à PostgreSQL"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Connected to PostgreSQL")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

conn = connect_db()

# ============================================================================
# CELL 3: Récupérer la dernière analyse IA
# ============================================================================

def get_latest_ai_analysis():
    """Récupère la dernière analyse IA"""
    query = """
    SELECT 
        analysis_timestamp,
        ai_provider,
        detections_count,
        affected_systems,
        critical_count,
        analysis_text
    FROM ai_threat_analysis
    ORDER BY analysis_timestamp DESC
    LIMIT 1
    """
    
    df = pd.read_sql_query(query, conn)
    
    if not df.empty:
        display(Markdown("## 🤖 Latest AI Analysis"))
        display(Markdown(f"**Generated:** {df['analysis_timestamp'].iloc[0]}"))
        display(Markdown(f"**Detections:** {df['detections_count'].iloc[0]}"))
        display(Markdown(f"**Critical:** {df['critical_count'].iloc[0]}"))
        display(Markdown("---"))
        display(Markdown(df['analysis_text'].iloc[0]))
    
    return df

ai_analysis = get_latest_ai_analysis()

# ============================================================================
# CELL 4: Identifier les systèmes à haut risque
# ============================================================================

def get_high_risk_systems(min_risk_score=300):
    """Identifie les systèmes nécessitant une intervention immédiate"""
    query = f"""
    WITH system_analysis AS (
        SELECT 
            agent_name,
            agent_ip,
            COUNT(DISTINCT technique_id) as unique_techniques,
            COUNT(*) as total_detections,
            SUM(CASE severity WHEN 'critical' THEN 1 ELSE 0 END) as critical_count,
            SUM(CASE severity 
                WHEN 'critical' THEN 100
                WHEN 'high' THEN 50
                WHEN 'medium' THEN 20
                WHEN 'low' THEN 5
            END) as severity_score,
            array_agg(DISTINCT technique_id) as techniques
        FROM apt41_detections
        WHERE timestamp >= NOW() - INTERVAL '24 hours'
        GROUP BY agent_name, agent_ip
    )
    SELECT 
        agent_name,
        agent_ip,
        unique_techniques,
        total_detections,
        critical_count,
        (unique_techniques * 10 + severity_score) as risk_score,
        techniques,
        CASE 
            WHEN (unique_techniques * 10 + severity_score) >= 500 
            THEN 'ISOLATE_IMMEDIATELY'
            WHEN (unique_techniques * 10 + severity_score) >= 300
            THEN 'INVESTIGATE_URGENT'
            ELSE 'MONITOR'
        END as action_required
    FROM system_analysis
    WHERE (unique_techniques * 10 + severity_score) >= {min_risk_score}
    ORDER BY risk_score DESC
    """
    
    df = pd.read_sql_query(query, conn)
    
    display(Markdown("## ⚠️ High Risk Systems Requiring Immediate Action"))
    display(df[['agent_name', 'agent_ip', 'risk_score', 'critical_count', 'action_required']])
    
    return df

high_risk_systems = get_high_risk_systems(300)

# ============================================================================
# CELL 5: Solutions par Technique APT41
# ============================================================================

REMEDIATION_PLAYBOOKS = {
    'T1550.002': {
        'name': 'Pass-the-Hash',
        'severity': 'CRITICAL',
        'immediate_actions': [
            'Isolate affected system from network',
            'Force password reset for all accounts on system',
            'Enable Credential Guard',
            'Disable NTLM authentication where possible',
            'Implement LAPS for local admin passwords'
        ],
        'powershell_commands': [
            # Activer Credential Guard
            'Set-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\Lsa" -Name "LsaCfgFlags" -Value 1',
            # Désactiver NTLM
            'Set-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\Lsa\\MSV1_0" -Name "NtlmMinClientSec" -Value 537395200',
            # Activer LSA Protection
            'Set-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\Lsa" -Name "RunAsPPL" -Value 1'
        ],
        'wazuh_rules': [
            'Restart Wazuh agent to apply new monitoring rules',
            'Enable enhanced credential monitoring'
        ]
    },
    'T1550.003': {
        'name': 'Pass-the-Ticket',
        'severity': 'CRITICAL',
        'immediate_actions': [
            'Reset KRBTGT password (twice, 10h apart)',
            'Revoke all active Kerberos tickets',
            'Enable Kerberos armoring (FAST)',
            'Audit all service accounts',
            'Check for Golden/Silver ticket indicators'
        ],
        'powershell_commands': [
            # Purger les tickets Kerberos
            'klist purge',
            # Activer Kerberos event logging
            'Set-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\Lsa\\Kerberos\\Parameters" -Name "LogLevel" -Value 1'
        ],
        'investigation_steps': [
            'Check for unusual TGT/TGS requests',
            'Review service account activity',
            'Search for mimikatz/rubeus artifacts'
        ]
    },
    'T1021.001': {
        'name': 'RDP Lateral Movement',
        'severity': 'HIGH',
        'immediate_actions': [
            'Review RDP connection sources',
            'Enable Network Level Authentication (NLA)',
            'Implement RDP gateway',
            'Restrict RDP access to specific IPs',
            'Enable RDP connection logging'
        ],
        'powershell_commands': [
            # Activer NLA
            '(Get-WmiObject -class Win32_TSGeneralSetting -Namespace root\\cimv2\\terminalservices -Filter "TerminalName=\'RDP-Tcp\'").SetUserAuthenticationRequired(1)',
            # Configurer le firewall
            'New-NetFirewallRule -DisplayName "RDP-Restricted" -Direction Inbound -LocalPort 3389 -Protocol TCP -Action Allow -RemoteAddress "10.0.0.0/8"'
        ]
    },
    'T1021.002': {
        'name': 'SMB/PsExec',
        'severity': 'CRITICAL',
        'immediate_actions': [
            'Verify PsExec usage authorization',
            'Check for malicious service installations',
            'Disable admin shares if not needed',
            'Implement Privileged Access Management',
            'Review SMB signing configuration'
        ],
        'powershell_commands': [
            # Désactiver les partages admin
            'net share ADMIN$ /delete',
            'net share C$ /delete',
            # Activer SMB signing
            'Set-SmbServerConfiguration -RequireSecuritySignature $true -Force',
            # Rechercher les services PSEXESVC
            'Get-Service | Where-Object {$_.Name -like "*PSEXE*"} | Stop-Service -Force'
        ]
    },
    'T1047': {
        'name': 'WMI Remote Execution',
        'severity': 'HIGH',
        'immediate_actions': [
            'Identify source system and user',
            'Verify legitimate administrative task',
            'Disable WMI if not required',
            'Monitor WMI event subscriptions',
            'Review WMI command execution logs'
        ],
        'powershell_commands': [
            # Lister les abonnements WMI suspects
            'Get-WMIObject -Namespace root\\Subscription -Class __EventFilter',
            'Get-WMIObject -Namespace root\\Subscription -Class CommandLineEventConsumer',
            # Désactiver WMI (si non utilisé)
            'Set-Service winmgmt -StartupType Disabled'
        ]
    },
    'T1003.001': {
        'name': 'LSASS Memory Dumping',
        'severity': 'CRITICAL',
        'immediate_actions': [
            'ISOLATE SYSTEM IMMEDIATELY',
            'Identify the accessing process',
            'Force password change for all users',
            'Enable LSA Protection',
            'Deploy Credential Guard',
            'Hunt for lateral movement'
        ],
        'powershell_commands': [
            # Activer LSA Protection
            'Set-ItemProperty -Path "HKLM:\\SYSTEM\\CurrentControlSet\\Control\\Lsa" -Name "RunAsPPL" -Value 1 -Type DWord',
            # Vérifier les processus ayant accédé à LSASS
            'Get-Process | Where-Object {$_.Name -eq "lsass"} | Select-Object -ExpandProperty Modules',
            # Redémarrer pour appliquer
            'Restart-Computer -Force'
        ]
    }
}

def display_remediation_guide(technique_id):
    """Affiche le guide de remédiation pour une technique"""
    if technique_id not in REMEDIATION_PLAYBOOKS:
        print(f"❌ No playbook found for {technique_id}")
        return
    
    playbook = REMEDIATION_PLAYBOOKS[technique_id]
    
    display(Markdown(f"# 🛡️ Remediation Guide: {playbook['name']}"))
    display(Markdown(f"**Severity:** {playbook['severity']}"))
    
    display(Markdown("## 🚨 Immediate Actions"))
    for i, action in enumerate(playbook['immediate_actions'], 1):
        display(Markdown(f"{i}. {action}"))
    
    if 'powershell_commands' in playbook:
        display(Markdown("## 💻 PowerShell Commands"))
        for cmd in playbook['powershell_commands']:
            display(Markdown(f"```powershell\n{cmd}\n```"))
    
    if 'investigation_steps' in playbook:
        display(Markdown("## 🔍 Investigation Steps"))
        for step in playbook['investigation_steps']:
            display(Markdown(f"- {step}"))

# Exemple d'utilisation
display_remediation_guide('T1550.002')

# ============================================================================
# CELL 6: Générer des scripts de remédiation automatiques
# ============================================================================

def generate_remediation_script(system_name, techniques):
    """Génère un script PowerShell de remédiation personnalisé"""
    
    script = f"""# ============================================================================
# APT41 Incident Response Script
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Target System: {system_name}
# Detected Techniques: {', '.join(techniques)}
# ============================================================================

# ÉTAPE 1: Collecte d'informations
Write-Host "============================================" -ForegroundColor Cyan
Write-Host "APT41 Incident Response - Starting" -ForegroundColor Cyan
Write-Host "System: {system_name}" -ForegroundColor Yellow
Write-Host "============================================" -ForegroundColor Cyan

# Créer un répertoire pour les logs
$logDir = "C:\\APT41_Response_$(Get-Date -Format 'yyyyMMdd_HHmmss')"
New-Item -Path $logDir -ItemType Directory -Force

# ÉTAPE 2: Capture de l'état actuel
Write-Host "[+] Capturing system state..." -ForegroundColor Green
Get-Process | Export-Csv "$logDir\\processes.csv" -NoTypeInformation
Get-Service | Export-Csv "$logDir\\services.csv" -NoTypeInformation
Get-NetTCPConnection | Export-Csv "$logDir\\network_connections.csv" -NoTypeInformation
Get-ScheduledTask | Export-Csv "$logDir\\scheduled_tasks.csv" -NoTypeInformation

"""
    
    # Ajouter les commandes spécifiques par technique
    for technique in techniques:
        if technique in REMEDIATION_PLAYBOOKS:
            playbook = REMEDIATION_PLAYBOOKS[technique]
            script += f"\n# ============================================================================\n"
            script += f"# Remediation for {technique}: {playbook['name']}\n"
            script += f"# ============================================================================\n\n"
            script += f"Write-Host '[+] Applying fixes for {playbook['name']}...' -ForegroundColor Yellow\n\n"
            
            for cmd in playbook.get('powershell_commands', []):
                script += f"try {{\n"
                script += f"    {cmd}\n"
                script += f"    Write-Host '  [OK] Command executed successfully' -ForegroundColor Green\n"
                script += f"}} catch {{\n"
                script += f"    Write-Host '  [ERROR] Command failed: $_' -ForegroundColor Red\n"
                script += f"    $_ | Out-File -Append '$logDir\\errors.log'\n"
                script += f"}}\n\n"
    
    script += """
# ÉTAPE FINALE: Rapport
Write-Host "============================================" -ForegroundColor Cyan
Write-Host "Remediation Complete" -ForegroundColor Green
Write-Host "Logs saved to: $logDir" -ForegroundColor Yellow
Write-Host "============================================" -ForegroundColor Cyan

# Créer un rapport
@"
APT41 Incident Response Report
===============================
Date: $(Get-Date)
System: {system_name}
Techniques Addressed: {', '.join(techniques)}

Actions Taken:
"""
    
    for technique in techniques:
        if technique in REMEDIATION_PLAYBOOKS:
            playbook = REMEDIATION_PLAYBOOKS[technique]
            script += f"- {playbook['name']}: Applied {len(playbook.get('powershell_commands', []))} fixes\n"
    
    script += """
Next Steps:
1. Review logs in $logDir
2. Monitor for 24 hours
3. Validate credential changes
4. Update Wazuh rules

"@ | Out-File "$logDir\\response_report.txt"

Write-Host "[+] Response report created" -ForegroundColor Green
"""
    
    return script

# Générer des scripts pour les systèmes à haut risque
for _, system in high_risk_systems.iterrows():
    script = generate_remediation_script(
        system['agent_name'],
        system['techniques']
    )
    
    # Sauvegarder le script
    filename = f"remediation_{system['agent_name'].replace(' ', '_')}.ps1"
    with open(filename, 'w') as f:
        f.write(script)
    
    print(f"✅ Generated: {filename}")

# ============================================================================
# CELL 7: Exécution à distance via Wazuh
# ============================================================================

def execute_wazuh_active_response(agent_id, command):
    """Exécute une commande via Wazuh Active Response"""
    
    # Configuration de l'active response
    active_response = {
        "command": "firewall-drop",  # ou "custom-script"
        "arguments": ["-"],
        "alert": {
            "rule": {
                "id": "100001"
            }
        }
    }
    
    print(f"🔧 Executing active response on agent {agent_id}")
    print(f"   Command: {command}")
    
    # Simulation (à adapter avec votre API Wazuh)
    print("   Status: Pending...")
    print("   ✅ Command sent successfully")

# Exemple d'utilisation
# execute_wazuh_active_response("001", "isolate-network")

# ============================================================================
# CELL 8: Surveillance post-remédiation
# ============================================================================

def monitor_post_remediation(agent_name, hours=24):
    """Surveille l'activité après remédiation"""
    
    query = f"""
    SELECT 
        timestamp AT TIME ZONE 'America/Toronto' as time,
        technique_id,
        technique_name,
        severity,
        rule_description
    FROM apt41_detections
    WHERE agent_name = '{agent_name}'
      AND timestamp >= NOW() - INTERVAL '{hours} hours'
    ORDER BY timestamp DESC
    """
    
    df = pd.read_sql_query(query, conn)
    
    display(Markdown(f"## 📊 Post-Remediation Monitoring: {agent_name}"))
    display(Markdown(f"**Period:** Last {hours} hours"))
    display(Markdown(f"**Total Detections:** {len(df)}"))
    
    if len(df) > 0:
        display(Markdown("### ⚠️ Persistent Activity Detected"))
        display(df)
        
        # Analyser si les mêmes techniques persistent
        persistent = df.groupby('technique_id').size().to_dict()
        display(Markdown("### Persistent Techniques:"))
        for tech, count in persistent.items():
            display(Markdown(f"- **{tech}**: {count} occurrences"))
    else:
        display(Markdown("### ✅ No suspicious activity detected"))
    
    return df

# Exemple d'utilisation
# monitor_post_remediation('WIN-SERVER-01', hours=6)

# ============================================================================
# CELL 9: Rapport d'incident complet
# ============================================================================

def generate_incident_report():
    """Génère un rapport d'incident complet"""
    
    report = f"""
# 🚨 APT41 Incident Response Report
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

## 📊 Executive Summary

"""
    
    # Statistiques globales
    stats_query = """
    SELECT 
        COUNT(DISTINCT agent_name) as affected_systems,
        COUNT(*) as total_detections,
        SUM(CASE WHEN severity = 'critical' THEN 1 ELSE 0 END) as critical_count,
        MIN(timestamp) as first_detection,
        MAX(timestamp) as last_detection
    FROM apt41_detections
    WHERE timestamp >= NOW() - INTERVAL '24 hours'
    """
    
    stats = pd.read_sql_query(stats_query, conn).iloc[0]
    
    report += f"""
- **Affected Systems:** {stats['affected_systems']}
- **Total Detections:** {stats['total_detections']}
- **Critical Alerts:** {stats['critical_count']}
- **First Detection:** {stats['first_detection']}
- **Last Detection:** {stats['last_detection']}

## 🎯 Detected Techniques
"""
    
    # Top techniques
    tech_query = """
    SELECT 
        technique_id,
        technique_name,
        COUNT(*) as count,
        MAX(severity) as max_severity
    FROM apt41_detections
    WHERE timestamp >= NOW() - INTERVAL '24 hours'
    GROUP BY technique_id, technique_name
    ORDER BY count DESC
    LIMIT 10
    """
    
    techniques = pd.read_sql_query(tech_query, conn)
    
    for _, tech in techniques.iterrows():
        report += f"\n### {tech['technique_id']}: {tech['technique_name']}\n"
        report += f"- **Occurrences:** {tech['count']}\n"
        report += f"- **Severity:** {tech['max_severity']}\n"
    
    report += "\n## 🛡️ Remediation Actions Taken\n"
    report += "\nSee individual system remediation scripts for details.\n"
    
    report += "\n## 📈 Recommendations\n"
    report += "\n1. Continue monitoring for 72 hours\n"
    report += "2. Conduct full network security audit\n"
    report += "3. Review and update incident response playbooks\n"
    report += "4. Schedule post-incident review meeting\n"
    
    # Sauvegarder le rapport
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"incident_report_{timestamp}.md"
    
    with open(filename, 'w') as f:
        f.write(report)
    
    display(Markdown(report))
    print(f"\n✅ Report saved: {filename}")
    
    return report

# Générer le rapport
incident_report = generate_incident_report()

# ============================================================================
# CELL 10: Export pour SIEM/SOAR
# ============================================================================

def export_for_siem():
    """Exporte les détections au format compatible SIEM"""
    
    query = """
    SELECT 
        timestamp,
        agent_name,
        agent_ip,
        technique_id,
        technique_name,
        severity,
        rule_id,
        rule_description,
        event_data
    FROM apt41_detections
    WHERE timestamp >= NOW() - INTERVAL '24 hours'
    ORDER BY timestamp DESC
    """
    
    df = pd.read_sql_query(query, conn)
    
    # Export CSV
    csv_file = f"apt41_detections_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df.to_csv(csv_file, index=False)
    print(f"✅ CSV exported: {csv_file}")
    
    # Export JSON pour SOAR
    json_file = f"apt41_detections_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    df.to_json(json_file, orient='records', date_format='iso')
    print(f"✅ JSON exported: {json_file}")
    
    return df

# Export
siem_data = export_for_siem()

print("\n✅ Notebook execution complete!")
print("📋 All remediation scripts and reports have been generated.")

✅ Connected to PostgreSQL
✅ Connected to PostgreSQL


## 🤖 Latest AI Analysis

**Generated:** 2025-12-09 00:03:33.764307

**Detections:** 9737

**Critical:** 5977

...

```powershell
Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\Lsa\MSV1_0" -Name "NtlmMinClientSec" -Value 537395200
```

```powershell
Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\Lsa" -Name "RunAsPPL" -Value 1
```

✅ Generated: remediation_SDC01VIRW22.ps1
✅ Generated: remediation_WIN11-C3.ps1



# 🚨 APT41 Incident Response Report
**Generated:** 2025-12-09 00:04:50

---

## 📊 Executive Summary


- **Affected Systems:** 2
- **Total Detections:** 9500
- **Critical Alerts:** 5893
- **First Detection:** 2025-12-08 00:05:36.487000
- **Last Detection:** 2025-12-08 23:38:13.314000

## 🎯 Detected Techniques

### T1021.002: SMB/PsExec
- **Occurrences:** 3186
- **Severity:** low

### T1550.003: Pass-the-Ticket
- **Occurrences:** 2965
- **Severity:** critical

### T1550.002: Pass-the-Hash
- **Occurrences:** 2954
- **Severity:** low

### T1021.001: RDP Lateral Movement
- **Occurrences:** 395
- **Severity:** low

## 🛡️ Remediation Actions Taken

See individual system remediation scripts for details.

## 📈 Recommendations

1. Continue monitoring for 72 hours
2. Conduct full network security audit
3. Review and update incident response playbooks
4. Schedule post-incident review meeting



✅ Report saved: incident_report_20251209_000450.md
✅ CSV exported: apt41_detections_20251209_000454.csv
✅ JSON exported: apt41_detections_20251209_000459.json

✅ Notebook execution complete!
📋 All remediation scripts and reports have been generated.
